In [ ]:
import pytesseract
import os
import random
import shutil
from tqdm import tqdm
from PIL import Image

In [ ]:
def convertToBinary(img):
    img = img.convert("RGB")
    pixdata = img.load()
    pixelCount = [ 0 for i in range(256)];
    pixelPro = [ 0 for i in range(256)];
    
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            red = pixdata[x, y][0] 
            green = pixdata[x, y][1]
            blue = pixdata[x, y][2]
            
            grey = (int)(red * 0.299 + green * 0.587 + blue * 0.114 + 0.5);
            pixelCount[grey] = pixelCount[grey]+1
            pixdata[x, y] = (grey, grey, grey, 255)

    for i in range(256):
        pixelPro[i] = pixelCount[i] * 1.0 / (img.size[1] * img.size[0])
        
    w0 , w1, u0tmp, u1tmp, u0, u1, deltaTmp, deltaMax,threshold = [0,0,0,0,0,0,0,0,127]
    for i in range(256):
        w0 = w1 = u0tmp = u1tmp = u0 = u1 = deltaTmp = 0 
        for j in range(256):
            if(j <= i): 
                w0 = w0 + pixelPro[j]
                u0tmp = u0tmp + j * pixelPro[j]
            else:
                w1 = w1 + pixelPro[j];  
                u1tmp = u1tmp + j * pixelPro[j]

        if w0 == 0:
            continue
        if w1 == 0:
            break
        u0 = u0tmp / w0
        u1 = u1tmp / w1
        deltaTmp = (float)(w0 *w1* pow((u0 - u1), 2))
        if(deltaTmp > deltaMax):
            deltaMax = deltaTmp
            threshold = i;  
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            if pixdata[x, y][0] <= threshold:
                pixdata[x, y] = (255, 255, 255, 255)
            else:
                pixdata[x, y] = (0, 0, 0, 255)
    return img

In [ ]:
def useTesseractToReg(input_path,output_path):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    for file in tqdm(os.listdir(input_path)):
        recoginze_digits = ''
        img = Image.open(os.path.join(input_path,file))
        img = img.resize(size=(28,28))
        
        # 2. the first recognize 
        first_recognize = pytesseract.image_to_string(img,config="-psm 7 digits")

        for digit in first_recognize:
            if digit >= '0' and digit <= '9':
                recoginze_digits = recoginze_digits + digit
            else:
                recoginze_digits='unrecognize'
                break
        
        #if the recognize is unknown or unrecognize , just skip this file
        if recoginze_digits == '' or recoginze_digits == 'unrecognize':
            continue
            
        # 3. convert the same file to white/black and recognize again
        img = convertToBinary(img)
        second_recognize = pytesseract.image_to_string(img,config="-psm 7 digits")
        
        # if the result of 1st and 2nd are the same , then we save the file
        # into a separate folder
        if second_recognize == first_recognize :
            if not os.path.exists(os.path.join(output_path,second_recognize)):
                os.mkdir(os.path.join(output_path,second_recognize))
            if not os.path.exists(os.path.join(output_path,second_recognize,file)):
                shutil.copy(os.path.join(input_path,file), 
                            os.path.join(output_path,second_recognize,file))

In [ ]:
if __name__ == '__main__':
    input_path = './datas/'
    output_path = './labeled/'
    
    # for single thread 
    useTesseractToReg(input_path,output_path)
    
    # for multi-thread to recognize files by folder
    '''
    threads = []
    sub_dirs = os.listdir(input_dirs)
    for sub_dir in sub_dirs:
        threads.append(threading.Thread(target=useTesseractToReg,args=(sub_dir,output_path,)))

    for t in threads:
        t.setDaemon(True)
        t.start()
    '''
